In [51]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv

In [52]:
df = pd.DataFrame({'title': [], 'text' : [], 'link': [], 'information_agency': []})

In [53]:
class Parser:
    def get_html(self, url):
        r = requests.get(url)
        r.encoding = 'utf8'
        return r.text
    
    def cat_link(self, s):
        return s.split('/')[2]
    
    def read_bbc(self, soup):
        try:
            main_story_body = soup.find('div', class_="story-body__inner").find_all('p') #bbc
            main_story_body = [i.string if i.string else " " for i in main_story_body]
            return ' '.join(main_story_body)
        except AttributeError:
            try:
                main_story_body = soup.find('div', class_="story-body__introdation").find_all('p') #bbc
                main_story_body = [i.string if i.string else " " for i in main_story_body]
                return ' '.join(main_story_body)
            except:
                return None
        
    def read_cnn(self, soup):
        try:
            main_story_body = soup.find('div', class_="l-container").find_all('div') #cnn
            main_story_body = [i.string if i.string else " " for i in main_story_body]
            return ' '.join(main_story_body)
        except AttributeError:
            return None
        
    def read_fake(self, soup):
        try:
            main_story_body1 = soup.find('div', class_="entry-content clearfix").find_all('h2') #fake
            main_story_body1 = [i.string if i.string else " " for i in main_story_body1]
            main_story_body2 = soup.find('div', class_="entry-content clearfix").find_all('p') #fake
            main_story_body2 = [i.string if i.string else " " for i in main_story_body2]
            main_story_body3 = soup.find('div', class_="entry-content clearfix").find_all('blockquote') #fake
            main_story_body3 = [i.string if i.string else " " for i in main_story_body3]
            main_story_body1 = ' '.join(main_story_body1)
            main_story_body2 = ' '.join(main_story_body2)
            main_story_body3 = ' '.join(main_story_body3)
            return ' '.join([main_story_body1, main_story_body2, main_story_body3])
        except AttributeError:
            return None
        
    def read_news(self, s, i):
        soup = BeautifulSoup(self.get_html(s), 'lxml')
        title = soup.find('title').string
        link = s
        information_agency = self.cat_link(link)
        text = None
        if information_agency == 'edition.cnn.com':
            text = self.read_cnn(soup)
        elif information_agency == 'www.bbc.co.uk':
            text = self.read_bbc(soup)
        elif information_agency == 'worldnewsdailyreport.com':
            text = self.read_fake(soup)
        data = {'title': title, 'text' : text, 'link': link, 'information_agency': information_agency}
        return pd.DataFrame(data, index=[i])

In [57]:
class get_data_bbc(get_data):
    def get_link_bbc(self, a):
        s = a.split('>')[0].split(' ')
        if len(s) == 1:
            return None
        else:
            return s[3].split('"')[1]
        
    def generate_link_bbc(self, a):
        links = []
        l = 'https://www.bbc.co.uk/search?q=covid&page='
        for i in range(1, a + 1):
            links.append(l + str(i))
        return links
    
    def read_news_bbc(self):
        df = pd.DataFrame({'title': [], 'text' : [], 'link': [], 'information_agency': []})
        for link_page in self.generate_link_bbc(29):
            soup = BeautifulSoup(self.get_html(link_page), 'lxml')
            head = soup.find('div', class_='css-1vg3h2e-Stack e1y4nx260')
            for i, j in zip(head, range(len(head))):
                    l = self.get_link_bbc(str(i.find('a')))
                    if l is None:
                        continue
                    else:
                        if l.split('/')[3] == 'news':
                            df = pd.concat([df, self.read_news(l,j)])
        return df

In [58]:
class get_data_fake(get_data):
    def generate_link_fake(self, a):
        links = []
        l = 'https://worldnewsdailyreport.com/category/health/disease-health/page/'
        for i in range(1, a + 1):
            links.append(l + str(i) + '/')
        return links
    
    def preprocessing_fake(self, h):
        h = [str(i) for i in h]
        h = [i.split('"')[1] for i in h]
        answer = []
        for i, number in zip(h, range(len(h))):
            if number % 2 == 1 :
                answer.append(i)
        del answer[len(answer) - 1]
        del answer[len(answer) - 1]
        return answer
    
    def read_news_fake(self):
        df = pd.DataFrame({'title': [], 'text' : [], 'link': [], 'information_agency': []})
        for link_page in self.generate_link_fake(4):
            soup = BeautifulSoup(self.get_html(link_page), 'lxml')
            head = []
            try:
                head = soup.find('div', id='main-content', class_='mh-loop').find_all('a')
            except:
                continue
            l_link = self.preprocessing_fake(head)
            for link, num in zip(l_link, range(len(l_link))):
                df = pd.concat([df, self.read_news(link,num)])
        return df

In [59]:
BBC = get_data_bbc()
BBC.read_news_bbc()

,title,text,link,information_agency
3,Momus writing an album inspired by Covid-19 - ...,Singer-songwriter Momus has nearly completed a...,https://www.bbc.co.uk/news/uk-scotland-52270653,www.bbc.co.uk
4,West Cornwall Hospital choir sings away Covid-...,None,https://www.bbc.co.uk/news/uk-england-cornwall...,www.bbc.co.uk
1,Coronavirus: Will Covid-19 speed up the use of...,"As a pandemic grips the world, a person could ...",https://www.bbc.co.uk/news/technology-52340651,www.bbc.co.uk
3,Coronavirus: Edinburgh NHS worker dies with Co...,A 73-year-old NHS worker in Edinburgh has died...,https://www.bbc.co.uk/news/uk-scotland-edinbur...,www.bbc.co.uk
6,Coronavirus: AI steps up in battle against Cov...,It feels as if a superhuman effort is needed t...,https://www.bbc.co.uk/news/technology-52120747,www.bbc.co.uk
7,Eurovision venue turned into Covid-19 hospital...,None,https://www.bbc.co.uk/news/world-europe-52325536,www.bbc.co.uk
9,Coronavirus: Carlisle stroke patient makes Cov...,A stroke patient who contracted coronavirus ha...,https://www.bbc.co.uk/news/uk-england-cumbria-...,www.bbc.co.uk
4,Sheffield student found in Coventry handed Cov...,A student who was stopped by police while atte...,https://www.bbc.co.uk/news/uk-england-coventry...,www.bbc.co.uk
5,GPs to trial Covid drugs to stop severe sympto...,Researchers have begun trials of drugs which c...,https://www.bbc.co.uk/news/health-52216240,www.bbc.co.uk
8,Covid-19: Police surgical masks 'provide no me...,The Scottish Police Federation has claimed new...,https://www.bbc.co.uk/news/uk-scotland-52319271,www.bbc.co.uk


In [56]:
FAKE_TAXI = get_data_fake()
FAKE_TAXI.read_news_fake()

,title,text,link,information_agency
0,COVID-19: 16 states report AR-15 shortages as ...,The very popular AR-15 style rifles sell like ...,https://worldnewsdailyreport.com/covid-19-16-s...,worldnewsdailyreport.com
1,Man who died of coronavirus miraculously resur...,A Sarasota resident who was declared clinicall...,https://worldnewsdailyreport.com/florida-man-w...,worldnewsdailyreport.com
2,San Francisco: Thousands infected by coronavir...,The San Francisco Department of Public Health ...,https://worldnewsdailyreport.com/san-francisco...,worldnewsdailyreport.com
3,Covid-19: Afghan government forces women to we...,Facing a shortage of disposable masks and a su...,https://worldnewsdailyreport.com/covid-19-afgh...,worldnewsdailyreport.com
4,Newly developed coronavirus test can detect vi...,A new test that can diagnose Covid-19 in just ...,https://worldnewsdailyreport.com/newly-develop...,worldnewsdailyreport.com
5,Warner Bros buys exclusive cinematographic rig...,The American entertainment conglomerate Warner...,https://worldnewsdailyreport.com/warner-bros-b...,worldnewsdailyreport.com
6,Mexico: Woman infected with COVID-19 gives bir...,A Mexican woman who has contracted the coronav...,https://worldnewsdailyreport.com/mexico-woman-...,worldnewsdailyreport.com
7,COVID-19: Chinese health authorities confirm p...,Chinese health authorities have confirmed the ...,https://worldnewsdailyreport.com/covid-19-chin...,worldnewsdailyreport.com
8,Spanish authorities ask pregnant women to abst...,The Spanish government announced new measures ...,https://worldnewsdailyreport.com/spanish-autho...,worldnewsdailyreport.com
9,"‘Coronavirus hasn’t changed my life a bit,’ sa...",A British man who has been living in a bubble ...,https://worldnewsdailyreport.com/coronavirus-h...,worldnewsdailyreport.com


In [ ]:
"""
with open("data.csv", 'a') as file:
        writer = csv.writer(file)
        writer.writerow(('title', 'text', 'link', 'information_agency'))

def csv_read(data):
    with open("data.csv", 'a') as file:
        writer = csv.writer(file)
        writer.writerow((data['title'], data['text'] ,data['link'], data['information_agency']))
"""

ModuleNotFoundError: No module named 'parser.Parser'; 'parser' is not a package